In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import Tensor

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Differentiating tensors
In part 01 we looked at how tensors can be used to store data and perform vectorised operations.
In PyTorch it is possible to differentiate the results of such operations with respect to the input variables, if desired.

To do this, the <mark>dependent variable (`Tensor`) must be initialised with `requires_grad` set to `True`</mark>. Then, anytime the tensor is used in an operation <mark>the gradient of that function will be included in the result, eventually allowing it to be differentiated</mark>.

In [11]:
dep_var = torch.tensor([6.0], requires_grad=True)
dep_var

tensor([6.], requires_grad=True)

In [17]:
data = torch.randn(1)
data

tensor([0.1603])

In [18]:
value = data*dep_var
value

tensor([0.9617], grad_fn=<MulBackward0>)

Note that the <mark>value has a `grad_fn`, which means it can be differentiated.</mark>

<mark>`torch.autograd` contains various functions to help with this, e.g. `torch.autograd.grad`:</mark>

In [19]:
from torch.autograd import grad

In [20]:
grad(outputs=value, inputs=dep_var) # differentiation of the value dep_var · data with respect to dep_var ( = data)

(tensor([0.1603]),)

### Differentiating functions
Above, we performed a function on a tensor and got the result, which we then differentiated with respect to the dependent input. <mark>It is also possible to differentiate functions directly for given inputs, without getting the results of the functions. I don't particularly like this.</mark>

Note that it also computes the gradient with respect to the data, even though the data wasn't set to require gradient.

These methods are <mark>mainly designed for *functional* programming, but you'd be better off looking into the [functorch ](https://pytorch.org/functorch/stable/) extension for PyTorch, or just using [JAX](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html)</mark>.

In [21]:
def func(data,dep_var):
    return data*dep_var

In [23]:
torch.autograd.functional.jacobian(func, (data,dep_var)) # derivative with respect to (data, dep_var) which gives (dep_var, data) of course

(tensor([[6.]]), tensor([[0.1603]]))

In [25]:
torch.autograd.functional.hessian(func, (data,dep_var)) # tensor of second derivatives ([[∂/∂data ∂data, ∂/∂data ∂dep_var], [∂/∂dep_var ∂data, ∂/∂dep_var ∂dep_var]]), which gives the antidiagonal matrix

((tensor([[0.]]), tensor([[1.]])), (tensor([[1.]]), tensor([[0.]])))

### Multiple variables and second derivative

In [26]:
dep_var_0 = torch.tensor([6.0], requires_grad=True)
dep_var_1 = torch.tensor([-2.0], requires_grad=True)

In [31]:
data = torch.randn(1)
data

tensor([-1.4519])

In [32]:
value = (dep_var_0**data)+dep_var_1.square()
value

tensor([4.0742], grad_fn=<AddBackward0>)

<mark>To differentiate w.r.t. multiple variables, include them as a tuple in the inputs</mark>. The <mark>`retain_graph` argument allows us to to recompute the gradient, if necessary, and the `create_graph` argument makes the output also have a `grad_fn`, is applicable</mark>, which allows to compute the second derivative, eventually.

In [33]:
jac = grad(outputs=value, inputs=(dep_var_0, dep_var_1), retain_graph=0, create_graph=True)
jac

(tensor([-0.0179], grad_fn=<WhereBackward0>),
 tensor([-4.], grad_fn=<MulBackward0>))

Since the gradient has a <mark>`grad_fn`, we can compute the second derivative, too. Note that we don't get the full Hessian matrix, though, only the diagonal.</mark>

In [35]:
grad(outputs=jac, inputs=(dep_var_0, dep_var_1), retain_graph=True, create_graph=True)

(tensor([0.0073], grad_fn=<WhereBackward0>),
 tensor([2.], grad_fn=<MulBackward0>))

#### Full Hessian

If you know that you'll be later computing the Hessian, or even just Jacobians, I find it best to have all the dependent variables in a single Tensor:

In [36]:
dep_vars = torch.tensor([6.0, -2.0], requires_grad=True)

In [37]:
value = (dep_vars[0]**data)+dep_vars[1].square()
value

tensor([4.0742], grad_fn=<AddBackward0>)

We compute the Jacobian as normal. This now returns the Jacobian in a single tensor, rather than a tuple of tensors.

In [38]:
jac = grad(outputs=value, inputs=dep_vars, retain_graph=True, create_graph=True)
jac

(tensor([-0.0179, -4.0000], grad_fn=<AddBackward0>),)

Now we try to compute the Hessian:

In [39]:
grad(outputs=jac, inputs=dep_vars, retain_graph=True, create_graph=True)

RuntimeError: grad can be implicitly created only for scalar outputs

Oh dear! The output needs to be a single value, and we provided two values. Instead we can supply each value in turn:

In [40]:
grad(outputs=jac[0][0], inputs=dep_vars, retain_graph=True, create_graph=True), grad(outputs=jac[0][1], inputs=dep_vars, retain_graph=True, create_graph=True)

((tensor([0.0073, 0.0000], grad_fn=<AddBackward0>),),
 (tensor([0., 2.], grad_fn=<AddBackward0>),))

<mark>So now we get the full Hessian matrix. (In this case the off-diagonals were zero, but they might not always be)</mark>

## Batched gradients and better Hessians
We already saw that the <mark>`grad` function has trouble dealing with non-scalar outputs, which meant we needed to call it twice and get a tuple</mark>. A more general way to do this, would be to <mark>iterate over each element of the Jacobian and stack the Hessian rows into a tensor</mark>:

In [41]:
torch.stack([grad(outputs=j, inputs=dep_vars, retain_graph=True, create_graph=True)[0] for j in jac[0].unbind(0)])  # unbind alows us to iterate through the tensor along the specified dimension

tensor([[0.0073, 0.0000],
        [0.0000, 2.0000]], grad_fn=<StackBackward0>)

In [42]:
%%timeit
torch.stack([grad(outputs=j, inputs=dep_vars, retain_graph=True, create_graph=True)[0] for j in jac[0].unbind(0)])  # unbind allows us to iterate through the tensor along the specified dimension

296 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


A slightly <mark>quicker way is to still feed in the full Jacobian, but use the `grad_outputs` to *switch on* which element we want to differentiate. `torch.eye` creates an identity matrix of a given size, and iterating through it will provide one-hot vectors.</mark>

In [43]:
torch.eye(len(jac[0]))

tensor([[1., 0.],
        [0., 1.]])

In [44]:
torch.stack([grad(outputs=jac[0], inputs=dep_vars, grad_outputs=i, retain_graph=True, create_graph=True)[0] for i in torch.eye(len(jac[0])).unbind(0)])

tensor([[0.0073, 0.0000],
        [0.0000, 2.0000]], grad_fn=<StackBackward0>)

In [45]:
%%timeit 
torch.stack([grad(outputs=jac[0], inputs=dep_vars, grad_outputs=i, retain_graph=True, create_graph=True)[0] for i in torch.eye(len(jac[0])).unbind(0)])

276 µs ± 6.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Vectorised method
The above method works, but still relies on a python for-loop to provide serial calls to `grad`. It would be better to instead <mark>perform all calls in parallel. We can do this using the PyTorch vmap function, although it is still experimental. It is only slightly faster.</mark>

`vmap` takes a function and a set of input arguments and will implicitly compute the function values by unbinding the inputs, and will then stack the results to a tensor:

In [49]:
from torch._vmap_internals import _vmap as vmap

In [50]:
vmap(lambda i: grad(outputs=jac[0], inputs=dep_vars, grad_outputs=i, retain_graph=True, create_graph=True)[0])(torch.eye(len(jac[0])))

tensor([[0.0073, 0.0000],
        [0.0000, 2.0000]], grad_fn=<AddBackward0>)

In [51]:
%%timeit
vmap(lambda i: grad(outputs=jac[0], inputs=dep_vars, grad_outputs=i, retain_graph=True, create_graph=True)[0])(torch.eye(len(jac[0])))

258 µs ± 6.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Batched gradients
The <mark>"non-scalar output" issue doesn't just apply to Hessians</mark>, what if we want to efficiently differentiate a batch of items independently?

In [52]:
dep_vars = torch.tensor([6.0, -2.0], requires_grad=True)

In [53]:
data = torch.rand(10,2)

In [54]:
values = (dep_vars[0]**data)+dep_vars[1].square()
values

tensor([[5.3601, 7.9987],
        [5.2061, 5.3302],
        [5.7386, 5.1194],
        [7.3961, 8.4607],
        [8.3072, 7.9371],
        [9.2684, 8.0694],
        [5.5813, 5.3362],
        [5.4156, 7.7234],
        [7.7857, 7.1862],
        [5.4934, 7.4062]], grad_fn=<AddBackward0>)

In [55]:
grad(outputs=values, inputs=dep_vars, retain_graph=True, create_graph=True)

RuntimeError: grad can be implicitly created only for scalar outputs

The <mark>trick is to reuse our vmap'd Jacobian, however even if we iterate over each row of the output values, it still isn't a scalar value; we need iterate over each element</mark>.

Rather than having the iteration adapt to every possible output shape, <mark>it is instead more convenient to write a generalised Jacobian function that works for any shape, by flattening and reshaping the inputs</mark>.

In [56]:
def jacobian(y: Tensor, x: Tensor, create_graph: bool = False, allow_unused: bool = True) -> Tensor:
    r"""
    Computes the Jacobian (dy/dx) of y with respect to variables x. x and y can have multiple elements.
    If y has multiple elements then computation is vectorised via vmap.

    Arguments:
        y: tensor to be differentiated
        x: dependent variables
        create_graph: If True, graph of the derivative will
            be constructed, allowing to compute higher order derivative products.
            Default: False.
        allow_unused: If False, specifying inputs that were not
            used when computing outputs (and therefore their grad is always

    Returns:
        dy/dx tensor of shape y.shape+x.shape
    """

    if len(y) == 0:
        return None
    flat_y = y.reshape(-1)

    def get_vjp(v: Tensor) -> Tensor:
        return torch.autograd.grad(flat_y, x, v, retain_graph=True, create_graph=create_graph, allow_unused=allow_unused)[0].reshape(x.shape)

    return vmap(get_vjp)(torch.eye(len(flat_y), device=y.device)).reshape(y.shape + x.shape)

In [57]:
jacobian(values, dep_vars)

tensor([[[ 0.0389, -4.0000],
         [ 0.5155, -4.0000]],

        [[ 0.0210, -4.0000],
         [ 0.0353, -4.0000]],

        [[ 0.0894, -4.0000],
         [ 0.0117, -4.0000]],

        [[ 0.3862, -4.0000],
         [ 0.6205, -4.0000]],

        [[ 0.5851, -4.0000],
         [ 0.5019, -4.0000]],

        [[ 0.8143, -4.0000],
         [ 0.5313, -4.0000]],

        [[ 0.0674, -4.0000],
         [ 0.0360, -4.0000]],

        [[ 0.0458, -4.0000],
         [ 0.4553, -4.0000]],

        [[ 0.4688, -4.0000],
         [ 0.3434, -4.0000]],

        [[ 0.0557, -4.0000],
         [ 0.3883, -4.0000]]])

In [58]:
jacobian(values, dep_vars).shape

torch.Size([10, 2, 2])

In [59]:
values.shape, dep_vars.shape

(torch.Size([10, 2]), torch.Size([2]))

This gives us the Jacobian of every element

## No-grad contexts
<mark>Calculations performed involving tensors with gradient incur an increased cost in terms of time and memory</mark>. In cases where gradient tracking isn't required, the <mark>context manager `no_grad` may be used:</mark>

In [60]:
dep_vars = torch.tensor([6.0, -2.0], requires_grad=True)

In [61]:
data = torch.rand(10,2)

In [62]:
%%timeit
(dep_vars[0]**data)+dep_vars[1].square()

15.6 µs ± 584 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [63]:
%%timeit
with torch.no_grad():
    (dep_vars[0]**data)+dep_vars[1].square()

10.9 µs ± 188 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


The <mark>`inference_mode` context is even more performant:</mark>

In [64]:
%%timeit
with torch.inference_mode():
    (dep_vars[0]**data)+dep_vars[1].square()

10.8 µs ± 591 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Modifying tensor with gradient
<mark>Once a tensor is set to have gradient, in-place modifications to it will result in an exception.</mark>

In [65]:
a = Tensor([3.0])

In [66]:
a[0] = 4.0

In [67]:
a.requires_grad = True

In [68]:
a[0] = 1.0

RuntimeError: a view of a leaf Variable that requires grad is being used in an in-place operation.

Here, the <mark>`no_grad` context can be used</mark>

In [70]:
with torch.no_grad():
    a[0] = 1.0
a

tensor([1.], requires_grad=True)

Or its `data` can be modified directly

In [71]:
a.data[0] = 7.0
a

tensor([7.], requires_grad=True)